In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 回测框架
import backtrader as bt

### 基础函数

In [2]:
#s_date = (datetime.datetime.now() - datetime.timedelta(days = 365))
#e_date = datetime.datetime.now()

def get_data_index(token, stock_code, s_date, e_date):
    ts.set_token(token)
    pro = ts.pro_api()
    df = pro.index_daily(ts_code=stock_code, 
                       fields='trade_date,open,high,low,close,vol',
                       start_date=s_date.strftime('%Y%m%d'), end_date=e_date.strftime('%Y%m%d'))
    df.index=pd.to_datetime(df.trade_date)
    df.drop('trade_date', axis=1, inplace = True)
    df = df.astype('float')
    df.rename(columns = {'vol':'volume'}, inplace = True)
    df['openinterest'] = 0
    return df.iloc[::-1]


def get_data_ts(token, stock_code, s_date, e_date):
    ts.set_token(token)
    pro = ts.pro_api()
    df = pro.daily(ts_code=stock_code, 
                       fields='trade_date,open,high,low,close,vol',
                       start_date=s_date.strftime('%Y%m%d'), end_date=e_date.strftime('%Y%m%d'))
    df.index=pd.to_datetime(df.trade_date)
    df.drop('trade_date', axis=1, inplace = True)
    df = df.astype('float')
    df.rename(columns = {'vol':'volume'}, inplace = True)
    df['openinterest'] = 0
    return df.iloc[::-1]

### 策略

In [32]:
class ATRStrategy_zz1000(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('initial', 10000),
        ('atr_period', 14),
        ('atr_percent', 1),
        ('atr_risk', 1),
        ('b_per', 20),
        ('b_dev', 2),
        ('pstake', 100),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            with open('log.txt', 'a') as file:
                file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        self.inds = dict()
        
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()

             # Add Bollinger Bands indicator (if index)
            self.inds[d]['linetop'] = bt.indicators.BollingerBands(d, 
                                                          period = self.params.b_per, 
                                                          devfactor = self.params.b_dev).top
            self.inds[d]['linebot'] = bt.indicators.BollingerBands(d, 
                                                          period = self.params.b_per, 
                                                          devfactor = self.params.b_dev).bot

            # Adding ATR indicators
            self.inds[d]['tr'] = bt.indicators.Max((d.high - d.low), 
                                                             abs(d.close(-1) - d.high),
                                                             abs(d.close(-1) - d.low))
            self.inds[d]['atr'] = bt.indicators.SimpleMovingAverage(self.inds[d]['tr'], period=self.params.atr_period)

        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
            else:  # Sell

                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        #self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        #if self.order:
        #    return
           
        # Bollinger Bands Buy Singal

        for d in self.datas:
            # Bollinger Bands Buy Signal
            if (d.close[0] > self.inds[d]['linetop'][0] and d.close[-1] < self.inds[d]['linetop'][-1]) or (d.close[0] > self.inds[d]['linebot'][0] and d.close[-1] <  self.inds[d]['linebot'][-1]):
#                self.log('BUY CREATE, Stock code:%s, Price: %.2f, Lots: %i, Current Position: %i' % (d._name, d.close[0], 
#                                                                                             100, self.getposition(d).size))
#                self.buy(data = d, size = 100, name = d._name)
                opt_pos = np.round(self.params.initial*self.params.atr_percent/100/self.inds[d]['atr'][0]/self.params.pstake)               
                if opt_pos * self.params.pstake > self.getposition(d).size:
                        # Keep track of the created order to avoid a 2nd order
                        #self.order = self.buy(size = opt_position - self.getposition(data).size)
                    self.log('BUY CREATE, Stock code:%s, Price: %.2f, Lots: %i, ATR: %.2f, Current Position: %i' % (d._name, d.close[0], 
                                                                                                                    opt_pos * self.params.pstake - self.getposition(d).size, self.inds[d]['atr'][0], self.getposition(d).size))
                    self.buy(data = d, size = opt_pos * self.params.pstake - self.getposition(d).size, name = d._name)
                   
                    
                elif opt_pos * self.params.pstake < self.getposition(d).size and self.getposition(d).size > 0:
                    self.log('SELL CREATE, Stock code:%s, Price: %.2f, Lots: %i, ATR: %.2f, Current Position: %i' % (d._name, d.close[0], 
                                                                                               self.getposition(d).size - opt_pos * self.params.pstake, self.inds[d]['atr'][0], self.getposition(d).size))
                    self.sell(data = d, size = self.getposition(d).size - opt_pos * self.params.pstake, name = d._name)
            # Bollinger Bands Sell Singal
            elif  (d.close[0] < self.inds[d]['linetop'][0] and d.close[-1] > self.inds[d]['linetop'][-1]) or (d.close[0] < self.inds[d]['linebot'][0] and d.close[-1] > self.inds[d]['linebot'][-1]):
                    #self.order = self.sell(data = d, price = d.close[0], size = self.getposition(d).size)
                    if self.getposition(d).size > 0:
                        self.log('SELL CREATE (Close), Stock code:%s, Price: %.2f, Lots: %i' % (d._name, d.close[0], 
                                                                                                self.getposition(d).size))
                        self.close(data = d, name = d._name)
                    
        # Keep track of the created order to avoid a 2nd order
        #self.order = self.sell(size = self.getposition(data).size - opt_position)                    


        
             
    def stop(self):
        self.log('Ending Value %.2f, Net Profit: %.2f%%' %
                 (self.broker.getvalue(), (self.broker.getvalue() / self.params.initial - 1) * 100), doprint = True)

##### 策略实验组
针对沪深300里面的每一只成分股，计算其(20, 2)布林带。由下向上突破布林带为买入信号，由上向下突破布林带为卖出信号。
+ 每次买入股数按照海龟ATR模型计算
+ 每次全部卖出

### 数据下载

In [117]:
import time

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)
stock_index = '000300.SH'
pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
index_list = np.unique(pro.index_weight(index_code=stock_index, 
                          start_date=s_date.strftime('%Y%m%d'), 
                          end_date=e_date.strftime('%Y%m%d')).con_code).tolist()

for s_code in index_list:
    df = get_data_ts('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                        s_code, s_date, e_date)
    time.sleep(1)
    df.to_csv('.\\Data\\' + s_code + '.csv')

### 回测

注：设置本金为5000000， 1ATR波动等同于总资金0.01%的波动是避免以下两种情况：
1. 资金不足无法执行买入信号。
2. 有买入信号，但是ATR计算后最优开仓不足100股导致不执行信号。

In [33]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(ATRStrategy_zz1000, printlog = False, initial = 5000000,  
                                 b_per = 20, b_dev = 2, atr_period = 14, atr_percent = 0.01) 

    s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
    e_date = datetime.date(2020, 12, 31)

    # Create stock Data Feed
    stock_index = '000300.SH'
    pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
    index_list = np.unique(pro.index_weight(index_code=stock_index, 
                              start_date=s_date.strftime('%Y%m%d'), 
                              end_date=e_date.strftime('%Y%m%d')).con_code).tolist()

    for s_code in index_list:
        df = pd.read_csv('.\\Data\\' + s_code + '.csv')
        df.index=pd.to_datetime(df.trade_date)
        data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

        # Add the index Data Feed to Cerebo
        cerebro.adddata(data, name = s_code)

    # Single stock testing 
#    df = get_data_ts('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
#                        '000001.SZ', s_date, e_date)
#    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)
#    cerebro.adddata(data, name = '600600.SH')
    
    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(5000000)

    # Set leverage
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()

2020-12-31: Ending Value 5187656.00, Net Profit: 3.75%
